In [6]:
# standard imports
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import glob
pd.set_option('display.max_colwidth',100)
import joblib
import pickle
import csv

# machine learning libraries
import sklearn            # machine-learning libary with many algorithms implemented
from sklearn.metrics import make_scorer
import xgboost as xgb     # extreme gradient boosting (XGB)
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Python file with supporting functions
import residual_utils as supporting_functions

import gcsfs
fs = gcsfs.GCSFileSystem()

2024-12-30 21:43:47.259169: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-30 21:43:47.270471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-30 21:43:47.284131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-30 21:43:47.288276: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 21:43:47.298558: I tensorflow/core/platform/cpu_feature_guar

<span style="color:hotpink; font-size:40px; font-weight:bold;">Setting date range</span>

In [7]:
# Define date range
date_range_start = '1982-02-01T00:00:00.000000000'
date_range_end = '2023-12-31T00:00:00.000000000'

# create date vector, adds 14 days to start & end
dates = pd.date_range(start=date_range_start, 
                      end=date_range_end,freq='MS')

init_date = str(dates[0].year) + format(dates[0].month,'02d')
fin_date = str(dates[-1].year) + format(dates[-1].month,'02d')

<span style="color:hotpink; font-size:40px; font-weight:bold;">Setting paths and choosing grid search approach</span>

<span style="color:lightblue; font-size:30px; font-weight:bold;">Grid search approach</span>

In [1]:
# for grid search, we can specify the metric to optimize for
# the options we can currently choose between are 'nmse' (negative mean square error) and 'bias' 

### DEFINE APPROACH HERE: ###
grid_search_approach = 'nmse'

<span style="color:lightblue; font-size:30px; font-weight:bold;">Paths</span>

In [ ]:
### set paths ###

your_username = # leap pangeo username, for bucket. should be your github username

### paths for loading: ###

# where machine learning inputs are saved
MLinputs_path = f"gs://leap-persistent/{your_username}/pco2_residual/post01_xgb_inputs"

path_seeds = "gs://leap-persistent/abbysh/pickles/random_seeds.npy" # random seeds for ML

#########################################

### paths for saving: ###

output_dir = f'gs://leap-persistent/{your_username}/pco2_residual/{grid_search_approach}/post02_xgb' # where to save machine learning results

model_output_dir = f"{output_dir}/trained" # where to save ML models
recon_output_dir = f"{output_dir}/reconstructions" # where to save ML reconstructions

metrics_output_dir = f'{output_dir}/metrics' # where to save performance metrics
test_perform_fname = f"{metrics_output_dir}/xgb_test_performance_{init_date}-{fin_date}.csv" # path for test performance metrics
unseen_perform_fname = f"{metrics_output_dir}/xgb_unseen_performance_{init_date}-{fin_date}.csv" # path for unseen performance metrics

xgb_model_save_dir = f'{output_dir}/saved_models_{init_date}-{fin_date}' # where to save .json model file

#########################################
jobs = 30

<span style="color:hotpink; font-size:40px; font-weight:bold;">Hyperparameter selection</span>

In [9]:
### Alternative Parameter grids, wider and more zoomed in ###

# stage one for wider grid:
# xg_param_grid = {"n_estimators":[50, 200, 500],
#                  "max_depth":[4, 10, None],
#                  ""
#                 }

# stage two zooming in
# xg_param_grid = {"n_estimators":[50, 500, 1000, 4000],
#                  "max_depth":[6, 7, 10, 15],
#                  "learning_rate":[0.05, 0.10, 0.30]
#                 }

In [10]:
# these are the parameters to try for grid search. These are all tried in combination with each other

xg_param_grid = {"n_estimators":[500, 1000, 2000, 4000],
                 "max_depth":[6, 7, 10, 15],
                 "learning_rate":[0.05, 0.10, 0.30, 0.40]
                }

In [14]:
# if you already know what ML parameters you want to use, define them in the dictionary "best_params_dict" below:

# best_params_dict = {"n_estimators": 4000,
#                  "max_depth": 6,
#                  "learning_rate": 0.30
#                 }

# if you've done grid search and the results are saved in a pickle file, and you want to use the results directly from the pickle, use "best_params_dict = None"
# the path is below in the XGBoost cell

best_params_dict = None

<span style="color:lightblue; font-size:30px; font-weight:bold;">To optimize for bias</span>

In [11]:
# if optimizing for bias, this is the function to score for low absolute value of bias.
# if optimizing for negative mean square error, this is not necessary 

def bias_fxn(truth,pred):
    bias = pred - truth
    return np.abs(np.nanmean(bias))

bias_scorer = make_scorer(bias_fxn, greater_is_better=False)

<span style="color:hotpink; font-size:40px; font-weight:bold;">Loading list of ESMs and members in testbed</span>

<span style="color:lightblue; font-size:30px; font-weight:bold;">For all members for each ESM</span>

In [9]:
### loads list of Earth System Models ("ensembles") and members for the full testbed ###

ensembles = []
for path in fs.ls(MLinputs_path):
    ens = path.split('/')[-1].split('.')[0]
    if ens not in ensembles:
        ensembles.append(ens)

mems_dict = dict()
a = fs.ls(MLinputs_path)
for ens_path in a:
    ens = ens_path.split('/')[-1]
    mems = fs.ls(ens_path)
    for mem in mems:
        memo = mem.split('/')[-1]
        
        if ens not in mems_dict:
            mems_dict[ens] = [memo]

        elif ens in mems_dict:
            mems_dict[ens].append(memo)

### loads random seeds for ML ###

random_seeds = np.load(fs.open(path_seeds))   

seed_loc_dict = defaultdict(dict)
for ens,mem_list in mems_dict.items():
    sub_dictt = dict()
    for no,mem in enumerate(mem_list):
        sub_dictt.update({mem:no})
    seed_loc_dict.update({ens:sub_dictt})

FileNotFoundError: b/leap-persistent/o/abbysh%2Fpco2_all_members_198202-202312%2Fpost01_xgb_inputs

<span style="color:lightblue; font-size:30px; font-weight:bold;">For one member per ESM</span>

In [ ]:
### if you only want one member per model ###

ensembles = []
for path in fs.ls(old_ensemble_dir):
    ens = path.split('/')[-1].split('.')[0]
    if ens not in ensembles:
        ensembles.append(ens)

mems_dict = dict()
a = fs.ls(old_ensemble_dir)
for ens_path in a:
    ens = ens_path.split('/')[-1]
    mems = fs.ls(ens_path)
    
    ### difference is here, only add one member per ensemble to dictionary
    for mem in mems[0:1]:
        memo = mem.split('/')[-1]
        
        if ens not in mems_dict:
            mems_dict[ens] = [memo]

        elif ens in mems_dict:
            mems_dict[ens].append(memo)

<span style="color:hotpink; font-size:40px; font-weight:bold;">Specifiying feature and target variables</span>

In [13]:
### Feature and target variables for use in the ML model ###

# features for ML:
features_sel = ['sst','sst_anom','sss','sss_anom','mld_clim_log','chl_log','chl_log_anom','xco2','A', 'B', 'C', 'T0', 'T1']

# the target variable we reconstruct:
target_sel = ['pco2_residual'] # this represents pCO2 - pCO2_T (calculated in notebook 00)

<span style="color:hotpink; font-size:40px; font-weight:bold;">Isolating training, testing, and validation sets</span>

We want to make sure we don't use the same data for training as we do for testing.
We also want to make sure that we don't use the same months from every year for either testing or training. 
For example, we don't want the month of May from each year to always be used for testing.

Therefore, the way we isolate the training and test sets is as follows:
Let's say we are using a testbed from January 1982 through December 2023. 
This is 504 months in total. Below, the code is counting 0 through 504.
If the number that the counting loop is at is divisible by 5, the associated month is used as a testing month. Otherwise, that month is used for training.
As a result, for each year in the testbed, two or three months will be used for testing and the rest for training. But, the specific months vary per year.

In [21]:
### train-validate-test split proportions ###
val_prop = .2 # 20% of data for validation
test_prop = .2 # 20% of data for testing

select_dates = []
test_dates = []

for i in range(0,len(dates)):
    
    if i % 5 != 0:
        select_dates.append(dates[i]) ### train days set ###
    if i % 5 == 0:
        test_dates.append(dates[i]) ### test days set ### 

### Then, the month numbers above are converted back to their respective datetime objects.

year_mon = []

for i in range(0,len(select_dates)):
    
    tmp = select_dates[i]
    year_mon.append(f"{tmp.year}-{tmp.month}")
    
test_year_mon = []

for i in range(0,len(test_dates)):
    
    tmp = test_dates[i]
    test_year_mon.append(f"{tmp.year}-{tmp.month}")

<span style="color:hotpink; font-size:40px; font-weight:bold;">Machine learning happens here</span>

In [ ]:
### "k_folds" means number of times to do cross validation. 
### When k_folds = 3, the same parameters are run through grid search 3 times, optimizing for your chosen approach (nmse or bias).
### The three scores are then averaged to get the overall score/performance for that combination of hyperparameters.

k_folds = 3

print(datetime.datetime.now())

for ens, mem_list in mems_dict.items():
    
    # If "first_mem = True", grid search will run for one member of each Earth System Model in the testbed.
    # If "first_mem = False", grid search will not run at all. 
        # Hyperparameters are then either: specified above in this notebook, or 
        # retrieved from a previously-made pickle file resulting from grid search.
    first_mem = True 

    for member in mem_list:
        print(ens,member)
        
        seed_loc = seed_loc_dict[ens][member]
    
        ### ML input path, a dataframe made in notebook 01 ###
        data_dir = f"{MLinputs_path}/{ens}/{member}"
        fname = f"MLinput_{ens}_{member.split('_')[-1]}_mon_1x1_{init_date}_{fin_date}.pkl"
        file_path = f"{data_dir}/{fname}"

        ### Read in ML input dataframe, create some selection filters, produce a reduced dataframe ###   
        with fs.open(file_path,'rb') as filee:
            df = pd.read_pickle(filee)
            df['year'] = df.index.get_level_values('time').year
            df['mon'] = df.index.get_level_values('time').month
            df['year_month'] = df['year'].astype(str) + "-" + df['mon'].astype(str)
            
            ### selects pCO2-residual values that correspond to SOCAT locations, 
            ### filtered to only use values between -250 and 250 microatm
            ### this more or less corresponds to filtering OUT values of pCO2 that are above 800 microatm, and therefore not realistic
            recon_sel = (~df[features_sel+target_sel+['net_mask']].isna().any(axis=1)) & ((df[target_sel] < 250) & (df[target_sel] > -250)).to_numpy().ravel()
            sel = (recon_sel & ((df['socat_mask'] == 1)))
            
            ### selection of training data, using filtered selection above, following train/test selection in cell above
            train_sel = (sel & (pd.Series(df['year_month']).isin(year_mon))).to_numpy().ravel()
            print('Number of training points:',sum(train_sel)) 

            ### selection of test data, using filtered selection above, following train/test selection in cell above
            test_sel = (sel & (pd.Series(df['year_month']).isin(test_year_mon))).to_numpy().ravel()
            print('Number of testing points:',sum(test_sel))

            ### selection of "unseen" data, corresponding to spatiotemporal points NOT in SOCAT (everything a "0" in the SOCAT mask)
            unseen_sel = (recon_sel & (df['socat_mask'] == 0))

            ### Convert dataframe of training and test data to numpy arrays
            X = df.loc[sel,features_sel].to_numpy()
            y = df.loc[sel,target_sel].to_numpy().ravel()

            ### Selects features and target variables from the training set
            Xtrain = df.loc[train_sel,features_sel].to_numpy()                
            ytrain = df.loc[train_sel,target_sel].to_numpy().ravel()

            ### Selects features and target variables from the test set
            X_test = df.loc[test_sel,features_sel].to_numpy()
            y_test = df.loc[test_sel,target_sel].to_numpy().ravel()  

            ### Applying splits in randomized order to training/test/validation sets using supporting python file ("supporting_functions")
            N = Xtrain.shape[0]
            train_val_idx, train_idx, val_idx, test_idx = supporting_functions.train_val_test_split(N, test_prop, val_prop, random_seeds, seed_loc)
            X_train_val, X_train, X_val, X_test_tmp, y_train_val, y_train, y_val, y_test_tmp = supporting_functions.apply_splits(Xtrain, ytrain, train_val_idx, train_idx, val_idx, test_idx)   

            
            ### GRID SEARCH STARTS HERE:
            ### Will optimize according to specified approach (bias or nmse)
            if first_mem:
                model = XGBRegressor(random_state=random_seeds[4,seed_loc], n_jobs=jobs)
                param_grid = xg_param_grid

                ### bias optimization
                if grid_search_approach == 'bias':
                    grid = GridSearchCV(model, param_grid, scoring=bias_scorer, cv=k_folds, return_train_score=True, refit=False, 
                            verbose=3)
                    
                ### nmse optimization
                elif grid_search_approach == 'nmse':
                    grid = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=k_folds, return_train_score=True, refit=False, 
                                    verbose=3)
                print(grid)
                ### Grid search here, using features and target variables from both training and validation sets, 
                    ### so should be using 80% of SOCAT sampling (60% train + 20% validation)
                grid.fit(X_train_val, y_train_val) 
                best_params = pd.DataFrame([grid.best_params_]) # best parameters chosen by grid search function due to best score
                print(best_params)
                scores = pd.DataFrame(grid.cv_results_)

                ### save grid search scores
                output_file = f"CVgrid_scores_{ens}_{member}.csv"
                output_dir = Path(f'{metrics_output_dir}')
                output_dir.mkdir(parents=True, exist_ok=True)
                scores.to_csv(output_dir / output_file)

                param_fname = f"{metrics_output_dir}/{grid_search_approach}_best_params_dict_{init_date}-{fin_date}_{ens}.pickle"

                ### saving grid search best parameters in pickle file
                best_params.to_pickle(param_fname)

                ### Here, "first_mem" is set to false so grid search stops after one member per ESM
                ### Do not change this to True UNLESS you want to do grid search on every member of the testbed
                first_mem = False
            ###### GRID SEARCH ENDS HERE ######
            
            ### make blank dictionaries for performance metrics for training/testing/unseen sets
            train_performance = defaultdict(dict)
            test_performance = defaultdict(dict)
            unseen_performance = defaultdict(dict)

            ### loading best hyperparameters from up above in notebook, if hyperparameters are hard-coded in notebook
            if best_params_dict is not None:
                best_params = best_params_dict

            ### loading previously-saved best hyperparameters from pickle file post-grid search
            elif best_params_dict is None:
                best_params_path = f"{metrics_output_dir}/{grid_search_approach}_best_params_dict_{init_date}-{fin_date}_{ens}.pickle"
                print(best_params_path)
                best_params = pd.read_pickle(best_params_path).to_dict('records')[0]
                print(best_params)

            ### Running ML using hyperparameters: setting up ML model and then training it (".fit") on training set
            model = XGBRegressor(random_state=random_seeds[5,seed_loc], **best_params, n_jobs=jobs)
            model.fit(X_train_val, y_train_val)          
            ### Save the resulting ML model
            supporting_functions.save_model(model, dates, xgb_model_save_dir, ens, member)
            
            ### Makes ML prediction/reconstruction on test set ML model to test set (".predict" and ".evaluate_test"), calculate test error metrics and store in a dictionary.
            ### X_test is the feature variables in the test set.
            ### Using the test set feature variables and the algorithm produced using the training data, 
                ### pCO2-residual (the target) is predicted.
            ### The predicted pCO2-residual within the test set domain is now compared to the TRUE test set pCO2-residual values.
            ### The result is the test performance.
            y_pred_test = model.predict(X_test)
            test_performance[ens][member] = supporting_functions.evaluate_test(y_test, y_pred_test)

            ### This saves the test performance metrics for every member in the testbed ###
            fields = test_performance[ens][member].keys()
            test_row_dict = dict()
            test_row_dict['model'] = ens
            test_row_dict['member'] = member
            
            for field in fields:
                test_row_dict[field] = test_performance[ens][member][field]

            file_exists = os.path.isfile(test_perform_fname)
            
            with open(test_perform_fname, 'a') as f_object:
                writer = csv.DictWriter(f_object, fieldnames = test_row_dict.keys())
                if not file_exists:
                    writer.writeheader() 
                writer.writerow(test_row_dict)

            print('test performance metrics:', test_performance[ens][member])
            
            ##############

            ### Global reconstruction where we don't have SOCAT data
            ### Then compares reconstruction to the testbed truth, which in our case is the pCO2-residual calculated from ESM output
            y_pred_unseen = model.predict(df.loc[unseen_sel,features_sel].to_numpy())
            y_unseen = df.loc[unseen_sel,target_sel].to_numpy().ravel()
            unseen_performance[ens][member] = supporting_functions.evaluate_test(y_unseen, y_pred_unseen)

            ### This saves the UNSEEN performance metrics for every member in the testbed ###
            fields = unseen_performance[ens][member].keys()
            unseen_row_dict = dict()
            unseen_row_dict['model'] = ens
            unseen_row_dict['member'] = member
            
            for field in fields:
                unseen_row_dict[field] = unseen_performance[ens][member][field]

            file_exists = os.path.isfile(unseen_perform_fname)

            with open(unseen_perform_fname, 'a') as f_object:
                writer = csv.DictWriter(f_object, fieldnames = unseen_row_dict.keys())
                if not file_exists:
                    writer.writeheader() 
                writer.writerow(unseen_row_dict)

            print('unseen performance metrics:', unseen_performance[ens][member])
            
            ##############
            
            # ML model predicts using feature variables from within SOCAT domain (should be all training and test locations/times)
            # This is in addition to the unseen prediction that's performed above
            y_pred_seen = model.predict(X)

            # Full reconstruction globally, every grid cell of testbed #
            # Unseen and "seen" (train + test) reconstructions are combined
            df['pCO2_recon_full'] = np.nan
            df.loc[unseen_sel,['pCO2_recon_full']] = y_pred_unseen 
            df.loc[sel,['pCO2_recon_full']] = y_pred_seen
            
            # Reconstruction in times/locations of TEST set domain, done by making all TRAINING points into nans
            df['pCO2_recon_test'] = np.nan
            df.loc[unseen_sel,['pCO2_recon_test']] = np.nan
            df.loc[sel,['pCO2_recon_test']] = y_pred_seen
            df.loc[train_sel, ['pCO2_recon_test']] = np.nan

            # Reconstructed in times/locations of TRAINING set domain, done by making all TESTING points into nans
            df['pCO2_recon_train'] = np.nan
            df.loc[unseen_sel,['pCO2_recon_train']] = np.nan
            df.loc[sel,['pCO2_recon_train']] = y_pred_seen
            df.loc[test_sel, ['pCO2_recon_train']] = np.nan
        
            # Reconstruction everywhere outside of SOCAT domain (this is what we call "unseen")
            df['pCO2_recon_unseen'] = np.nan
            df.loc[unseen_sel,['pCO2_recon_unseen']] = y_pred_unseen
            df.loc[sel,['pCO2_recon_unseen']] = np.nan
            
            # Reconstruction at time/locations of SOCAT sampling (train + test)
            df['pCO2_recon_socat'] = np.nan
            df.loc[unseen_sel,['pCO2_recon_socat']] = np.nan
            df.loc[sel,['pCO2_recon_socat']] = y_pred_seen

            # Testbed truth (pco2 residual calculated from ESM output)
            df['pCO2_residual_testbed_truth'] = df['pco2_residual']

            DS_recon = df[['net_mask','socat_mask','pCO2_residual_testbed_truth','pCO2_recon_full','pCO2_recon_socat','pCO2_recon_unseen', 'pCO2_recon_test', 'pCO2_recon_train']].to_xarray()

            supporting_functions.save_recon(DS_recon, dates, recon_output_dir, ens, member)
            print('end of member', datetime.datetime.now())
            
print('end of all members', datetime.datetime.now())

2024-11-21 04:29:12.044935
ACCESS-ESM1-5 member_r4i1p1f1
238718
58855
/home/jovyan/grid_search_test/saved_models_1982-2022_nmse_test/xg_best_params_dict_1982-2022_ACCESS-ESM1-5_member_r4i1p1f1.pickle
{'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 4000}
Starting model saving process
Save complete
test performance metrics: {'mse': 69.45649136902385, 'mae': 5.293047913915207, 'medae': 3.4573132106062303, 'max_error': 172.88574063996077, 'bias': 0.16168219666709582, 'r2': 0.9293837437086994, 'corr': 0.9647763243382422, 'cent_rmse': 8.332487629497209, 'stdev': 29.09101, 'amp_ratio': 0.7955313918087857, 'stdev_ref': 31.362023475346902, 'range_ref': 445.85411982281875, 'iqr_ref': 35.39009281133522}
unseen performance metrics: {'mse': 120.11119129551156, 'mae': 6.917735108534175, 'medae': 4.585168425708957, 'max_error': 265.2407128060157, 'bias': 0.9918781548554838, 'r2': 0.8751175797253312, 'corr': 0.93693370755052, 'cent_rmse': 10.914548443128798, 'stdev': 27.776644, 'amp_ratio': 0

<span style="color:hotpink; font-size:40px; font-weight:bold;">To view ranking of hyperparameters and their various scores</span>

In [ ]:
# scoresCV = scores[['params', 'mean_test_score', 'std_test_score', 'mean_train_score']].sort_values(by = 'mean_test_score' \
#                    , ascending = False) #mean = averaged over the k folds (in this case, the 3 partitions of the cross validation); std = standard deviation

# scoresCV = scores[['params', 'mean_test_score', 'std_test_score', 'mean_fit_time']].sort_values(by = 'mean_test_score' \
#                     , ascending = False) #mean = averaged over the k folds (in this case, the 3 partitions of the cross validation); std = standard deviation

# scoresCV.head(10)